In [10]:
from nilmtk.dataset_converters import convert_redd
convert_redd('redd/low_freq', 'learnRedd.h5')


Loading house 1... 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
Loading house 2... 1 2 3 4 5 6 7 8 9 10 11 
Loading house 3... 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 
Loading house 4... 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
Loading house 5... 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 
Loading house 6... 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 
Loaded metadata
Done converting YAML metadata to HDF5!
Done converting REDD to HDF5!


In [18]:
import pandas as pd
import numpy as np
from nilmtk import DataSet

In [70]:

class RNNDisaggregator():

    def __init__(self):
        self.mmax = None
        
    def train(self, mains, meter, epochs=1, batch_size=128, **load_kwargs):
        '''Train
        Parameters
        ----------
        mains : a nilmtk.ElecMeter object for the aggregate data
        meter : a nilmtk.ElecMeter object for the meter data
        epochs : number of epochs to train
        batch_size : size of batch used for training
        **load_kwargs : keyword arguments passed to `meter.power_series()`
        '''

        main_power_series = mains.power_series( **load_kwargs) 'generator object next() print hoga'
        print("",load_kwargs)
        print("MAIN POWER********",main_power_series)
        meter_power_series = meter.power_series(**load_kwargs)
        print("METER POWER********",meter_power_series)

        # Train chunks
        run = True
        mainchunk = next(main_power_series)
        meterchunk = next(meter_power_series)
        print("**************************** TRAIN********************")
        
        print("Mainchunk train :",mainchunk.max())
        print("meterchunk train :",meterchunk.max())
        if self.mmax == None:
            self.mmax = mainchunk.max()

        while(run):
            mainchunk = self._normalize(mainchunk, self.mmax)
            meterchunk = self._normalize(meterchunk, self.mmax)

            self.train_on_chunk(mainchunk, meterchunk, epochs, batch_size)
            try:
                mainchunk = next(main_power_series)
                meterchunk = next(meter_power_series)
            except:
                 run = False
   
    def _normalize(self, chunk, mmax):
        '''Normalizes timeseries
        Parameters
        ----------
        chunk : the timeseries to normalize
        max : max value of the powerseries
        Returns: Normalized timeseries
        '''
        print("**********************In Normalize***********")
        print("******chunk :",chunk)
        print("******mmax :",mmax )
        tchunk = chunk / mmax
        print("******tchunk :", tchunk)
        return tchunk
    def train_on_chunk(self, mainchunk, meterchunk, epochs, batch_size):
        '''Train using only one chunk
        Parameters
        ----------
        mainchunk : chunk of site meter
        meterchunk : chunk of appliance
        epochs : number of epochs for training
        batch_size : size of batch used for training
        '''

        # Replace NaNs with 0s
        mainchunk.fillna(0, inplace=True)
        meterchunk.fillna(0, inplace=True)
        ix = mainchunk.index.intersection(meterchunk.index)
        print("ix*****" ,ix)
        mainchunk = np.array(mainchunk[ix])
        meterchunk = np.array(meterchunk[ix])

        mainchunk = np.reshape(mainchunk, (mainchunk.shape[0],1,1))
        print("main chunk*********" ,mainchunk)

In [65]:
train = DataSet('learnRedd.h5')
train.set_window(end="30-4-2011") #Use data only until 4/30/2011
train_elec = train.buildings[1].elec

In [66]:
train_mains = train_elec.mains().all_meters()[1] # The aggregated meter that provides the input
train_meter = train_elec.submeters()['electric oven'] # The microwave meter that is used as a training target

In [67]:
Myrnn = RNNDisaggregator()

In [92]:
Myrnn.train(train_mains, train_meter, epochs=1, sample_period=3)

 {'sample_period': 3}
MAIN POWER******** <generator object Electric.load_series at 0x7f386d132830>
METER POWER******** <generator object Electric.load_series at 0x7f386d132bf8>
Loading data for meter ElecMeterID(instance=4, building=1, dataset='REDD')     
Done loading data all meters for this chunk.
**************************** TRAIN********************
Mainchunk train : 4068.343505859375
meterchunk train : 4272.5
**********************In Normalize***********
******chunk : 2011-04-18 09:22:09-04:00     118.889999
2011-04-18 09:22:12-04:00     118.820000
2011-04-18 09:22:15-04:00     119.120003
2011-04-18 09:22:18-04:00     118.976662
2011-04-18 09:22:21-04:00     119.150002
                                ...     
2011-04-29 23:59:45-04:00    1186.703369
2011-04-29 23:59:48-04:00    1185.799927
2011-04-29 23:59:51-04:00    1184.439941
2011-04-29 23:59:54-04:00    1185.453369
2011-04-29 23:59:57-04:00    1185.043335
Freq: 3S, Name: (power, apparent), Length: 334357, dtype: float32
****